In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import anndata as ad
import scanpy as sc

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.colors as clr

import scvi
import squidpy as sq
import cellcharter as cc
import gc

c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\docrep\decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\docrep\decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and 

In [2]:
sc.settings.n_jobs = -1

In [3]:
import torch
torch.set_float32_matmul_precision('high')
torch.cuda.is_available()

True

In [4]:
from lightning.pytorch import seed_everything
seed_everything(12345)

Seed set to 12345


12345

In [5]:
import shapely as sp 
import geopandas as gpd
import libpysal as lps 
import esda
import alphashape as ash

In [6]:
neighborhood_colors = {
    "DEJ": "#0000FF",
    "PERIVASC-I": "#FF0000",
    "DIFF-IFE": "#00FF00",
    "PERIVASC-II": "#000033",
    "STROMA": "#FF00B6",
    "UPPER-HF": "#005300",
    "ECCRINE": "#FFD300",
    "SEB-GLND": "#009FFF",
    "SUBCUTIS": "#9A4D42",
    "LOWER-HF": "#00FFBE"
}

In [7]:
def get_boundary(coords):
    x = coords.get_coordinates().to_numpy()
    poly = lps.cg.alpha_shape_auto(x, step=1)
    return poly

In [8]:
adata = sc.read_h5ad("../../data/merfish/BAYSOR/seurat_objects/ns-atlas.merfish_baysor.scanvi_integrated.cellcharter.anndata.annotated.h5ad")
adata = adata[adata.obs['cell_type.detailed'] != "Unknown" & adata.obs['cell_type.detailed'] != "Doublet" & adata.obs['tissue_compartment'] != 'outside tissue'].copy()

In [9]:
adata.obs.head(10)

,EntityID,fov,volume,center_x,center_y,min_x,min_y,max_x,max_y,anisotropy,...,cell_id,tissue_compartment,compartment.short,sample_compartment,donor_age,race_ethnicity,DV200,RIN,preservation_type,collection_type
1577244300000000000,1577244300000000000,NaN,413.582367,10833.988067,5659.833278,10830.405562,5656.014926,10838.326891,5662.925291,1.535899,...,D151_SKIN_NS_S04_R01_C1577244300000000000,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000001,1577244300000000001,NaN,954.476268,11088.754365,5967.087526,11081.775322,5960.465239,11095.173641,5971.335980,1.820862,...,D151_SKIN_NS_S04_R01_C1577244300000000001,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000002,1577244300000000002,NaN,2314.160266,9533.604246,6785.310602,9526.056136,6778.549632,9543.619093,6794.195938,1.315076,...,D151_SKIN_NS_S04_R01_C1577244300000000002,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000003,1577244300000000003,NaN,1377.588139,10330.515975,5700.319847,10320.137869,5693.969074,10343.894569,5705.211938,3.583034,...,D151_SKIN_NS_S04_R01_C1577244300000000003,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000006,1577244300000000006,NaN,364.332251,11125.854334,6012.214286,11122.113357,6009.566081,11131.581724,6014.614107,1.857338,...,D151_SKIN_NS_S04_R01_C1577244300000000006,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000008,1577244300000000008,NaN,1137.968435,10640.326353,7268.846994,10634.916613,7262.627689,10647.572479,7274.467481,1.099862,...,D151_SKIN_NS_S04_R01_C1577244300000000008,epidermis,EPI,D151_SKIN_NS_S04_R01_EPI,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000013,1577244300000000013,NaN,471.085336,11066.605192,5821.276310,11061.106763,5816.906358,11072.093556,5825.109851,1.461268,...,D151_SKIN_NS_S04_R01_C1577244300000000013,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000014,1577244300000000014,NaN,965.433572,10854.263308,5504.434883,10848.463487,5499.716736,10859.962040,5509.464750,1.076560,...,D151_SKIN_NS_S04_R01_C1577244300000000014,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000015,1577244300000000015,NaN,514.916523,9348.450583,5482.567898,9344.775658,5477.369742,9351.470938,5487.585487,1.468110,...,D151_SKIN_NS_S04_R01_C1577244300000000015,dermis,DER,D151_SKIN_NS_S04_R01_DER,36.0,NaN,0.52,NaN,FFPE,autopsy
1577244300000000017,1577244300000000017,NaN,1179.214414,8472.042450,5949.888550,8467.573480,5941.958979,8477.315849,5958.837520,1.936929,...,D151_SKIN_NS_S04_R01_C1577244300000000017,epidermis,EPI,D151_SKIN_NS_S04_R01_EPI,36.0,NaN,0.52,NaN,FFPE,autopsy


## Spatial Proximity: Cell Types

In [6]:
## cell type proximity enrichment
ct_proximity_df = cc.gr.nhood_enrichment(
    adata, cluster_key='cell_type.detailed',
    symmetric=True, observed_expected=True,
    log_fold_change=True, only_inter=False,
    pvalues=True, n_perms=1000,
    copy=True, n_jobs=12
    )

100%|██████████| 1000/1000 [18:32<00:00,  1.11s/it] 
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


In [7]:
adata.uns['cell_type.detailed_nhood_enrichment'] = ct_proximity_df

In [9]:
ct_proximity_df['enrichment'].to_csv("../../data/merfish/BAYSOR/spatial_proximity/all_samples.cell_type.detailed.proximity_enrichment.results.enrichment_zscores.csv")
ct_proximity_df['pvalue'].to_csv("../../data/merfish/BAYSOR/spatial_proximity/all_samples.cell_type.detailed.proximity_enrichment.results.enrichment_pvalues.csv")
ct_proximity_df['observed'].to_csv("../../data/merfish/BAYSOR/spatial_proximity/all_samples.cell_type.detailed.proximity_enrichment.results.enrichment_observed.csv")
ct_proximity_df['expected'].to_csv("../../data/merfish/BAYSOR/spatial_proximity/all_samples.cell_type.detailed.proximity_enrichment.results.enrichment_expected.csv")